In [10]:
import json 
import os 
import subprocess
import numpy as np 
import shutil
from munch import Munch
import yaml
import copy
import time

In [11]:
src = "/home/tianqi/in-context-learning-main/models/linear_regression/pretrained_probing" # training result path

config_path = "/home/tianqi/in-context-learning-main/src/conf/probing/linear_regression_probing_template.yaml" # config file template path

total_layer = 5 # total number of layers
with open(config_path, "r") as fp:  # we don't Quinfig it to avoid inherits
    base_conf = Munch.fromDict(yaml.safe_load(fp))

In [12]:
print(base_conf)

Munch({'model': Munch({'family': 'lstm', 'n_embd': 512, 'n_layer': 5, 'n_head': 8, 'n_dims': 20, 'n_positions': 101, 'p_dropout': 0, 'has_p_embedding': False, 'use_first_n_layer': 1}), 'training': Munch({'train_steps': 10001, 'resume_id': 'probing_1', 'task': 'linear_regression', 'data': 'gaussian', 'task_kwargs': Munch({}), 'batch_size': 64, 'learning_rate': 0.0001, 'save_every_steps': 1000, 'keep_every_steps': 100000, 'curriculum': Munch({'dims': Munch({'start': 20, 'end': 20, 'inc': 1, 'interval': 2000}), 'points': Munch({'start': 41, 'end': 41, 'inc': 2, 'interval': 2000})})}), 'out_dir': '../models/by_layer_linear/', 'wandb': Munch({'name': 'linear_regression_probing_1', 'project': 'in-context-training-probtest', 'entity': 'tianqi_chen', 'notes': None, 'log_every_steps': 100})})


In [13]:
# create folder for each layer
for i in range(1, total_layer + 1):
    dst = f"/home/tianqi/in-context-learning-main/models/by_layer_linear/probing_{i}"
    print(f"finish create folder for probing_{i}")
    if not os.path.exists(dst):
        shutil.copytree(src, dst) 

finish create folder for probing_1
finish create folder for probing_2
finish create folder for probing_3
finish create folder for probing_4
finish create folder for probing_5


In [14]:
base_conf

Munch({'model': Munch({'family': 'lstm', 'n_embd': 512, 'n_layer': 5, 'n_head': 8, 'n_dims': 20, 'n_positions': 101, 'p_dropout': 0, 'has_p_embedding': False, 'use_first_n_layer': 1}), 'training': Munch({'train_steps': 10001, 'resume_id': 'probing_1', 'task': 'linear_regression', 'data': 'gaussian', 'task_kwargs': Munch({}), 'batch_size': 64, 'learning_rate': 0.0001, 'save_every_steps': 1000, 'keep_every_steps': 100000, 'curriculum': Munch({'dims': Munch({'start': 20, 'end': 20, 'inc': 1, 'interval': 2000}), 'points': Munch({'start': 41, 'end': 41, 'inc': 2, 'interval': 2000})})}), 'out_dir': '../models/by_layer_linear/', 'wandb': Munch({'name': 'linear_regression_probing_1', 'project': 'in-context-training-probtest', 'entity': 'tianqi_chen', 'notes': None, 'log_every_steps': 100})})

In [15]:
# generate config files for each layer
for i in range(1, total_layer + 1):
    probe_i_conf = copy.deepcopy(base_conf)

    probe_i_conf['model']['use_first_n_layer'] = i
    probe_i_conf['training']['resume_id'] = f'probing_{i}'
    # probe_i_conf['training']['data'] = 'skew'
    probe_i_conf['wandb']['name'] = f"linear_regression_probing_lstm_{i}"

    probe_i_conf_path = f"/home/tianqi/in-context-learning-main/src/conf/probing/linear_regression_probing_{i}.yaml"
    with open(probe_i_conf_path, "w") as fp:  # we don't Quinfig it to avoid inherits
        #base_conf = Munch.fromDict(yaml.safe_load(fp))
        yaml.safe_dump(probe_i_conf, fp)

In [20]:
for i in range(1, total_layer + 1):
    with open("tmp.sh", "w") as fp:
        data = f"""#!/bin/bash

#SBATCH --output=slurm_output/%j.log # Output and error log path

source ~/.bashrc
source activate /home/tianqi/anaconda3/envs/in-context-learning

"""

        data += f"python train_linear.py --config conf/probing/linear_regression_probing_{i}.yaml"

        fp.write(data)
        os.chmod("tmp.sh", 0o0777)
    
    subprocess.Popen("sbatch --gres=gpu:a6000:1 --time 1-0 tmp.sh", shell=True, stdout=subprocess.PIPE)
    print(f"submitted for probing {i}")
    time.sleep(60)

submitted for probing 1
submitted for probing 2


KeyboardInterrupt: 